# Data Structures - List Variants
###### (from "Algorithm Design with Haskell")

Lets look at a data structure that optimizes certain operations for which the regular List type performs poorly.

### Symmetric List

In [1]:
newtype SymList a = SymList { unSymList :: ([a], [a]) } deriving (Show, Eq, Ord)

This data strucure respresents a List using the following abstracting function

In [3]:
fromSL :: SymList a -> [a]
fromSL (SymList (xs, ys)) = xs ++ reverse ys

There are several invariants that operations on `SymList` will take advantage of and maintain.

`null xs => null ys || single ys`

`null ys => null xs || single xs`

In [8]:
snocSL :: a -> SymList a -> SymList a
snocSL a (SymList (xs, ys))
  | null xs = SymList (ys, [a]) -- we know ys is either empty or a singleton.
  | otherwise = SymList (xs, a : ys)
  
consSL :: a -> SymList a -> SymList a
consSL a (SymList (xs, ys)) = SymList (a : xs, ys)
  
lastSL :: SymList a -> a
lastSL (SymList (xs, ys))
  | null ys = head xs
  | otherwise = head ys
  
headSL :: SymList a -> a
headSL (SymList (xs, ys))
  | null xs = head ys
  | otherwise = head xs
  
tailSL :: SymList a -> SymList a
tailSL (SymList (xs, ys))
  | null xs || null ys = emptySL
  | length xs > 1 = SymList (tail xs, ys)
  | otherwise = SymList (reverse vs, us)
  where
    (us, vs) = splitAt (length ys `div` 2) ys
    
initSL :: SymList a -> SymList a
initSL (SymList (xs, ys))
  | null xs || null ys = emptySL
  | length ys > 1 = SymList (xs, tail ys)
  | otherwise = SymList (us, reverse vs)
  where
    (us, vs) = splitAt (length xs `div` 2) xs
  
emptySL :: SymList a
emptySL = SymList ([], [])

We can use the symmetric list to define `inits` such that `length . inits` takes linear time. It is an online function meaning it works over infinite lists.

In [13]:
inits :: [a] -> [[a]]
inits = map fromSL . scanl (flip snocSL) emptySL

### Random-access Lists
We can use a binary tree to build a structure that supports logarithmic operations for cons, snoc, tail, and indexing.

In [17]:
data Tree a
  = Leaf a
  | Node Word (Tree a) (Tree a)
  deriving (Show, Eq, Ord)
  
size :: Tree a -> Word
size (Leaf _) = 1
size (Node l _ _) = l

node :: Tree a -> Tree a -> Tree a
node t1 t2 = Node (size t1 + size t2) t1 t2

The Trees we will work with will always be "perfect" meaning that they have sizes of the form 2^p for p >= 0. This will garuanteed by the operations.

We will arrange a list of perfect Trees such that the sizes of those trees correspond to a binary number in reverse order. So we represent 6 elements as 011 = [Zero, One (Node 2 ...), One (Node 4 ...].

Here is the definition of the random-access list:

In [37]:
data Digit a = Zero | One (Tree a)
type RAList a = [Digit a]

fromRA :: RAList a -> [a]
fromRA = concatMap from where
  from Zero = []
  from (One t) = fromT t
  fromT (Leaf a) = [a]
  fromT (Node _ t1 t2) = foldr (:) (fromT t2) $ fromT t1
  
fetchRA :: Word -> RAList a -> a
fetchRA k (Zero : xs) = fetchRA k xs
fetchRA k (One t : xs)
  | k < size t = fetchT k t
  | otherwise = fetchRA (k - size t) xs
  where
    fetchT 0 (Leaf a) = a
    fetchT k (Node s l r)
      | k < m = fetchT k l
      | otherwise = fetchT (k - m) r
      where m = s `div` 2
      
nullRA :: RAList a -> Bool
nullRA = null

nilRA :: RAList a
nilRA = []

updateRA :: Word -> a -> RAList a -> RAList a
updateRA _ _ [] = []
updateRA k x (Zero : xs) = Zero : updateRA k x xs
updateRA k x (One t : xs)
  | k < size t = One (updateT k x t) : xs
  | otherwise = One t : updateRA (k - size t) x xs
  where
    updateT 0 x (Leaf _) = Leaf x
    updateT k x (Node s t1 t2)
      | k < m = Node s (updateT k x t1) t2
      | otherwise = Node s t1 (updateT (k - m) x t2)
      where m = s `div` 2
      
consRA :: a -> RAList a -> RAList a
consRA x xs = consT (Leaf x) xs where
  consT t [] = [One t]
  consT t (Zero : xs) = One t : xs
  consT t1 (One t2 : xs) = Zero : consT (node t1 t2) xs
  
unconsRA :: RAList a -> (a, RAList a)
unconsRA xs = (x, ys) where
  (Leaf x, ys) = unconsT xs
  unconsT (One t : xs) = if null xs then (t, []) else (t, Zero : xs)
  unconsT (Zero : xs) = (t1, One t2 : ys) where (Node _ t1 t2, ys) = unconsT xs 
  
headRA :: RAList a -> a
headRA = fst . unconsRA

tailRA :: RAList a -> RAList a
tailRA = snd . unconsRA